# Env
Install some dependencies

In [1]:
# reinstall kaggle new version
! pip install --upgrade --force-reinstall --no-deps kaggle

In [2]:
# install ipdb for debugging
! pip install ipdb

# Mount Google Drive
Mount google drive and set path for loading and saving both data and model

In [1]:
import os
from google.colab import drive

In [2]:
# mount drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# set project path
project_path = 'dl-hw1'

DEV_PATH = '/content/drive/MyDrive/CMU/dev'
PRJ_PATH = os.path.join(DEV_PATH, project_path)
INITDATA_PATH = os.path.join(DEV_PATH, project_path, 'data')
MODEL_PATH = os.path.join(DEV_PATH, project_path, 'model')
LOG_PATH = os.path.join(DEV_PATH, project_path, 'log')

In [4]:
# create project dir
def mkdir(p):
    if os.path.isdir(p) is False:
        os.mkdir(p)
        print("Create dir: {}".format(p))

mkdir(PRJ_PATH)
mkdir(INITDATA_PATH)
mkdir(MODEL_PATH)
mkdir(LOG_PATH)

Create dir: /content/drive/MyDrive/CMU/dev/dl-hw1
Create dir: /content/drive/MyDrive/CMU/dev/dl-hw1/data
Create dir: /content/drive/MyDrive/CMU/dev/dl-hw1/model
Create dir: /content/drive/MyDrive/CMU/dev/dl-hw1/log


# Load data
Load data from Kaggle

## Initial download
from Kaggle

In [ ]:
import json

In [ ]:
# copy kaggle authentication
kaggle_file = os.path.join(DEV_PATH, '.kaggle/kaggle.json')
! mkdir /root/.kaggle
! cp $kaggle_file /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

# set default dir
! kaggle config set -n path -v $INITDATA_PATH

usage: kaggle config set [-h] -n NAME -v VALUE
kaggle config set: error: argument -v/--value: expected one argument


In [ ]:
# download competition data
! kaggle competitions download -c 11-785-s22-hw1p2

100% 1.85G/1.86G [00:23<00:00, 105MB/s]
100% 1.86G/1.86G [00:23<00:00, 86.4MB/s]


## Load existing data
from Google Drive

In [3]:
# load data from drive
zip_file = kaggle_file = os.path.join(INITDATA_PATH, 'competitions/11-785-s22-hw1p2/11-785-s22-hw1p2.zip')
! unzip $zip_file

# Tensorboard
Setup Tensorboard to monitor performance

In [ ]:
# setup tensorboard login
! tensorboard dev list

# upload during training (run in console)
# tensorboard dev upload --logdir /content/drive/MyDrive/CMU/dev/dl-hw1/log --description "hw1p2"


***** TensorBoard Uploader *****

This will list all experiments that you've uploaded to
https://tensorboard.dev. TensorBoard.dev experiments are visible
to everyone. Do not upload sensitive data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=UdCJ5SDzdm1EV97xKa6OKps440bS63&prompt=consent&access_type=offline
Enter the authorization c

# Setup
Setup script for 
- Data loading
- Model training, validating, testing
- Model saving, running, resuming, predicting

In [6]:
# load packages
import os
import datetime
import pytz
import time
import csv
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import tensorboard
from sklearn.metrics import accuracy_score

In [7]:
# Dataset
class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None):
        '''
        Batch download files
            data_path: data path
            sample: how many npy files will be preloaded for one __getitem__ call
            shuffle: shuffle or not
            partition: directory to load file
            csvpath: test by csvfile
        '''
        self.sample = sample 
        
        # find x and y directory
        self.X_dir = os.path.join(data_path, partition, 'mfcc')
        self.Y_dir = os.path.join(data_path, partition, 'transcript')
        
        # find all files in the directory
        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]
        
        # random files in the path
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
        
        # find amount of file
        assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        
        # set y label
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>'] # start of sentence / end of sentence
    
    # for loading csv
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    # for getting the length
    def __len__(self):
        return int(np.ceil(self.length / self.sample))
    
    # call when getting one file data
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        # load data for each np file
        X, Y = [], []
        for j in sample_range:
            X_path = os.path.join(self.X_dir, self.X_names[j])
            Y_path = os.path.join(self.Y_dir, self.Y_names[j])
            
            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))
        
        # keep data separate for each file
        X, Y = np.array(X, dtype=object), np.array(Y, dtype=object)
        return X, Y

# Dataloader
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context=0):
        '''
        Load data by item
            X (list of array): batch data of features
            Y (list of array): batch data of label
            context: specify number of data before and after item
        '''

        # define data index mapping
        index_map_X = []
        for i, x in enumerate(X):
            for j, xx in enumerate(x):
                index_map_X.append((i, j))
        
        # define label index mapping
        index_map_Y = []
        for i, y in enumerate(Y):
            for j, yy in enumerate(y):
                index_map_Y.append((i, j))
        
        # store variable
        assert(set(index_map_X) == set(index_map_Y))
        self.length = len(index_map_X)
        self.index_map = index_map_X
        self.context = context

        if context == 0:
            # no padding
            self.X, self.Y = X, Y
        else:
            # pad 0 before and after each file
            x = []
            for i in range(len(X)):
                x.append(np.pad(X[i], [(context, context), (0, 0)], mode='constant', constant_values=0))
            self.X = np.array(x, dtype=object)
            self.Y = Y
    
    # get the length
    def __len__(self):
        return self.length
    
    # get data
    def __getitem__(self, index):
        # get data by index
        i, j = self.index_map[index]
        if self.context == 0:
            x = self.X[i][j,:].flatten()
            y = self.Y[i][j]
        else:
            x = self.X[i][j:(j + self.context*2 + 1),:].flatten()
            y = self.Y[i][j]
        return x, y

# Dataset for test
class LibriTest(torch.utils.data.Dataset):
    def __init__(self, data_path, partition="test-clean", test_order='test_order.csv', context = 0):
        '''
        Load data without label
            data_path: data path
            partition: test directory
            test_order: competition file order
            context: specify number of data before and after item
        '''
        self.X_dir = os.path.join(data_path, partition, 'mfcc')
        self.X_names = list(pd.read_csv(test_order)['file'])
        
        # load data for each np file
        X = []
        for j in range(len(self.X_names)):
            X_path = os.path.join(self.X_dir, self.X_names[j])
            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
        X = np.array(X, dtype=object)

        # define data index mapping
        index_map_X = []
        for i, x in enumerate(X):
            for j, xx in enumerate(x):
                index_map_X.append((i, j))
        self.length = len(index_map_X)
        self.index_map = index_map_X
        self.context = context

        if context == 0:
            # no padding
            self.X = X
        else:
            # pad 0 before and after each file
            x = []
            for i in range(len(X)):
                x.append(np.pad(X[i], [(context, context), (0, 0)], mode='constant', constant_values=0))
            self.X = np.array(x, dtype=object)

    # get length
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        # get data by index
        i, j = self.index_map[index]
        if self.context == 0:
            x = self.X[i][j,:].flatten()
        else:
            x = self.X[i][j:(j + self.context*2 + 1),:].flatten()
        return x


In [8]:
# Train / Val / Test

def train(args, model, device, train_samples, optimizer, criterion, epoch):
    '''
    Train data from training dataset
    '''

    model.train()
    train_loss = []

    # run for each training sample
    for i in range(len(train_samples)):
        # load dataset and dataloader
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], 
                                                   shuffle=True, num_workers=args['num_workers'])
        
        # run by catch
        for batch_idx, (data, target) in enumerate(train_loader):
            # load data to device (cuda)
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)

            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())

            # print loss 
            if batch_idx % args['log_interval'] == 0:
                print('Train Epoch: {} \tBatch: {}/{}[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, i+1, len(train_samples), 
                    batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
    
    # return average training loss
    return np.mean(train_loss)

def val(args, model, device, dev_samples, criterion):
    '''
    Validate data from validated dataset
    '''

    model.eval()
    true_y_list = []
    pred_y_list = []
    val_loss = []

    with torch.no_grad():
        # loop for each sample
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]
            # load dataset and dataloader
            val_items = LibriItems(X, Y, context=args['context'])
            val_loader = torch.utils.data.DataLoader(val_items, batch_size=args['batch_size'], 
                                                     shuffle=False, num_workers=args['num_workers'])
            # predict
            for data, true_y in val_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                loss = criterion(output, true_y)
                val_loss.append(loss.item())

                pred_y = torch.argmax(output, axis=1)
                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())

    # return average validation loss and accuracy
    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    return np.mean(val_loss), train_accuracy

def test(test_data, model, device, args):
    '''
    Predict data from test dataset
    '''

    model.eval()
    pred_y_list = []

    with torch.no_grad():
        # load dataloader
        test_loader = torch.utils.data.DataLoader(test_data, batch_size=args['batch_size'], 
                                                  shuffle=False, num_workers=args['num_workers'])

        # loop for each item and predict
        for data in test_loader:
            data = data.float().to(device)             
            
            output = model(data)
            pred_y = torch.argmax(output, axis=1)

            pred_y_list.extend(pred_y.tolist())

    return pred_y_list

In [9]:
# Run model

# save model to the path every epoch (last model and best model)
def save_model(exp_name, model_path, model, optimizer, epoch, metrics, total_time, best_model=False):
    # set experiment path
    exp_path = os.path.join(model_path, exp_name)
    if os.path.isdir(exp_path) is False:
        os.mkdir(exp_path)
        print("Create dir: {}".format(exp_path))
    # save last model and metrics to run.tar
    exp_run_path = os.path.join(exp_path, 'run.tar')
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': metrics[0][-1],
        'val_loss': metrics[1][-1],
        'val_acc': metrics[2][-1],
        'metrics': metrics,
        'total_time': total_time
        }
    torch.save(checkpoint, exp_run_path)
    print("Save model: {}".format(exp_run_path))
    # if this is the best model save model and metrics to model.tar
    if best_model:
        exp_best_path = os.path.join(exp_path, 'model.tar')
        torch.save(checkpoint, exp_best_path)
        print("Save model: {}".format(exp_best_path))

# run training
def run(model, optimizer, criterion, args, viz=False):
    # set experiment name
    exp_name = args['exp_name'] + '_' + datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y%m%d-%H%M%S")
    # set tensorboard name
    if viz:
        writer = tensorboard.SummaryWriter(os.path.join(args['log_path'],  exp_name))
    
    # check device
    print('Using device:', next(model.parameters()).device)

    # load batch dataset
    train_samples = LibriSamples(data_path = args['data_path'], sample=args['sample'], shuffle=True, partition="train-clean-100", csvpath=args.get('csvpath'))
    val_samples = LibriSamples(data_path = args['data_path'], shuffle=True, partition="dev-clean")

    # loop by epoch
    best_loss = 0
    trn_loss_list = []
    val_loss_list = []
    val_acc_list = []
    start_time = time.time()
    for epoch in range(1, args['epoch'] + 1):
        # get loss and accuracy value
        print('#### EPOCH {} ####'.format(epoch))
        trn_loss = train(args, model, device, train_samples, optimizer, criterion, epoch)
        val_loss, val_acc = val(args, model, device, val_samples, criterion)

        # append data to list
        print('** train loss={:.6f} | validation loss={:.6f} | validation accuracy={:.2f}%'.format(
            trn_loss, val_loss, val_acc*100.
            ))
        trn_loss_list.append(trn_loss)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)
        metrics = np.array([trn_loss_list, val_loss_list, val_acc_list])

        # write data to tensor board
        if viz:
            writer.add_scalar("Loss/train", trn_loss, epoch)
            writer.add_scalar("Loss/val", val_loss, epoch)
            writer.add_scalar("Accuracy/val", val_acc, epoch)

        # test best model
        if epoch == 1:
            best_model = True
            best_loss = val_loss
        elif val_loss < best_loss:
            best_model = True
            best_loss = val_loss
        else:
            best_model = False
        total_time = time.time() - start_time

        # save model
        save_model(exp_name, args['model_path'], model, optimizer, epoch, metrics, total_time, best_model)
    
    # close writer
    if viz:
        writer.flush()

# resume training from the last model
def resume(model, optimizer, criterion, args, viz=False):
    # load model
    checkpoint = torch.load(os.path.join(args['model_path'], args['exp_load'], 'run.tar'))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    metrics = checkpoint['metrics']

    # set experiment name
    exp_name = args['exp_name'] + '_' + datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y%m%d-%H%M%S")

    # write existing metrics
    if viz:
        writer = tensorboard.SummaryWriter(os.path.join(args['log_path'],  exp_name))
        for i in range(metrics.shape[1]):
            writer.add_scalar("Loss/train", metrics[0][i], i+1)
            writer.add_scalar("Loss/val", metrics[1][i], i+1)
            writer.add_scalar("Accuracy/val", metrics[2][i], i+1)
    
    # load batch dataset
    train_samples = LibriSamples(data_path = args['data_path'], sample=args['sample'], shuffle=True, partition="train-clean-100", csvpath=args.get('csvpath'))
    val_samples = LibriSamples(data_path = args['data_path'], shuffle=True, partition="dev-clean")

    # loop by epoch
    best_loss = np.min(metrics[1])
    trn_loss_list = list(metrics[0])
    val_loss_list = list(metrics[1])
    val_acc_list = list(metrics[2])
    start_time = time.time()
    for epoch in range(checkpoint['epoch'] + 1, args['epoch'] + 1):
        # get loss and accuracy value
        print('#### EPOCH {} ####'.format(epoch))
        trn_loss = train(args, model, device, train_samples, optimizer, criterion, epoch)
        val_loss, val_acc = val(args, model, device, val_samples, criterion)

        # append data to list
        print('** train loss={:.6f} | validation loss={:.6f} | validation accuracy={:.2f}%'.format(
            trn_loss, val_loss, val_acc*100.
            ))
        trn_loss_list.append(trn_loss)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)
        metrics = np.array([trn_loss_list, val_loss_list, val_acc_list])

        # write data to tensor board
        if viz:
            writer.add_scalar("Loss/train", trn_loss, epoch)
            writer.add_scalar("Loss/val", val_loss, epoch)
            writer.add_scalar("Accuracy/val", val_acc, epoch)

        # test best model
        if epoch == checkpoint['epoch'] + 1:
            best_model = True
            best_loss = val_loss
        elif val_loss < best_loss:
            best_model = True
            best_loss = val_loss
        else:
            best_model = False
        total_time = time.time() - start_time + checkpoint['total_time']

        # save model
        save_model(exp_name, args['model_path'], model, optimizer, epoch, metrics, total_time, best_model)
    
    # close writer
    if viz:
        writer.flush()

# predict model from best model (model.tar)
def pred_test(model, device, args):
    # load model
    checkpoint = torch.load(os.path.join(args['model_path'], args['exp_load'], 'model.tar'))
    model.load_state_dict(checkpoint['model_state_dict'])
    print('predict from model: {} (train loss={:.6f} | validation loss={:.6f} | validation accuracy={:.2f}% | epoch={})'.format(
        args['exp_load'], checkpoint['train_loss'], checkpoint['val_loss'], checkpoint['val_acc']*100, checkpoint['epoch']
        ))
    
    # load test dataset
    test_data = LibriTest(data_path = args['data_path'], context=args['context'])
    y_pred = test(test_data, model, device, args)

    # write to csv
    df = pd.DataFrame({
        "id": range(len(y_pred)),
        "label": y_pred
    })
    df.to_csv(args['file_out'], index=False)


# Model Architecture
All model architectures that are already trained
- The best model is Network08
- Exclude 01-04 which haven't saved in this script

In [10]:
class Network05(torch.nn.Module):
    '''
    Pyramid architecture
    Default, no context
    '''
    def __init__(self, in_size):
        super(Network05, self).__init__()
        # in_size = 13
        layers = [
            nn.Linear(in_size, 2048),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 40),
        ]
        self.layers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.layers(A0)
        return x

class Network06(torch.nn.Module):
    '''
    Pyramid architecture
    Increase size of the first layer
    '''
    def __init__(self, in_size):
        super(Network06, self).__init__()
        # in_size = 13
        layers = [
            nn.Linear(in_size, 4096),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 40),
        ]
        self.layers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.layers(A0)
        return x

class Network07(torch.nn.Module):
    '''
    Pyramid architecture
    Add context
    Reduce layers
    '''
    def __init__(self, context):
        super(Network07, self).__init__()
        # context = 50
        in_size = 13*(args['context']*2+1)
        layers = [
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, 2048),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 40),
        ]
        self.layers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.layers(A0)
        return x


class Network08(torch.nn.Module):
    '''
    Cylindrical architecture
    Remain context
    '''
    def __init__(self, context):
        super(Network08, self).__init__()
        # context = 50
        in_size = 13*(args['context']*2+1)
        layers = [
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, in_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(in_size),
            nn.Linear(in_size, 40),
        ]
        self.layers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.layers(A0)
        return x

# Run / Resume

## Run
Train new experiment

In [ ]:
'''
Argument variables
    Batch size: batch size to load for each batch
    context: relevant context around x to load for prediction
    log_interval: interval for printing each log
    data_path: path to get data
    lr: learning rate
    epoch: how many epochs to run for this experiment
    exp_name: experiment name to save
    sample: how many files to load for each loop
    num_workers: number of workers to run
    model_path: path of the model
    log_path: path of the log file
'''

args = {
    'batch_size': 2048,
    'context': 50,
    'log_interval': 1000,
    'data_path': '/content/hw1p2_student_data',
    'lr': 0.001,
    'epoch': 100,
    'exp_name': 'layer_08',
    'sample': 10000,
    'num_workers': 2,
    'model_path': MODEL_PATH,
    'log_path': LOG_PATH,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set model architecture
model = Network08(args['context']).to(device)
optimizer = optim.Adam(model.parameters(), lr=args['lr'])
criterion = torch.nn.CrossEntropyLoss()

run(model, optimizer, criterion, args, viz=True)

## Resume
Resume existing model to continue training

In [ ]:
# Resume
'''
Argument variables
    Batch size: batch size to load for each batch
    context: relevant context around x to load for prediction
    log_interval: interval for printing each log
    data_path: path to get data
    lr: learning rate
    epoch: how many epochs to run for this experiment
    exp_name: experiment name to save
    exp_load: experiment name to load when resuming model
    sample: how many files to load for each loop
    num_workers: number of workers to run
    model_path: path of the model
    log_path: path of the log file
'''

args = {
    'batch_size': 2048,
    'context': 50,
    'log_interval': 1000,
    'data_path': '/content/hw1p2_student_data',
    'lr': 0.001,
    'epoch': 100,
    'exp_name': 'layer_08',
    'exp_load': 'layer_08_20220208-012704',
    'sample': 10000,
    'num_workers': 2,
    'model_path': MODEL_PATH,
    'log_path': LOG_PATH,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Network08(args['context']).to(device)
optimizer = optim.Adam(model.parameters(), lr=args['lr'])
criterion = torch.nn.CrossEntropyLoss()

#run(model, optimizer, criterion, args, viz=True)
resume(model, optimizer, criterion, args, viz=True)

#### EPOCH 82 ####
Train Epoch: 82 	Batch: 1/3[0/12677455 (0%)]	Loss: 0.362907
Train Epoch: 82 	Batch: 1/3[2048000/12677455 (16%)]	Loss: 0.371115
Train Epoch: 82 	Batch: 1/3[4096000/12677455 (32%)]	Loss: 0.367860
Train Epoch: 82 	Batch: 1/3[6144000/12677455 (48%)]	Loss: 0.372688
Train Epoch: 82 	Batch: 1/3[8192000/12677455 (65%)]	Loss: 0.389365
Train Epoch: 82 	Batch: 1/3[10240000/12677455 (81%)]	Loss: 0.400370
Train Epoch: 82 	Batch: 1/3[12288000/12677455 (97%)]	Loss: 0.354776
Train Epoch: 82 	Batch: 2/3[0/12663079 (0%)]	Loss: 0.418745
Train Epoch: 82 	Batch: 2/3[2048000/12663079 (16%)]	Loss: 0.378728
Train Epoch: 82 	Batch: 2/3[4096000/12663079 (32%)]	Loss: 0.376037
Train Epoch: 82 	Batch: 2/3[6144000/12663079 (49%)]	Loss: 0.425448
Train Epoch: 82 	Batch: 2/3[8192000/12663079 (65%)]	Loss: 0.418343
Train Epoch: 82 	Batch: 2/3[10240000/12663079 (81%)]	Loss: 0.397052
Train Epoch: 82 	Batch: 2/3[12288000/12663079 (97%)]	Loss: 0.421043
Train Epoch: 82 	Batch: 3/3[0/10850600 (0%)]	Loss: 0.

# Predict

## Predict
Predict test dataset to output file

In [13]:
'''
Argument variables
    Batch size: batch size to load for each batch
    context: relevant context around x to load for prediction
    data_path: path to get data
    exp_load: experiment name to load when resuming model
    sample: how many files to load for each loop
    num_workers: number of workers to run
    model_path: path of the model
    file_out: output file name
'''

args = {
    'batch_size': 2048,
    'context': 50,
    'data_path': '/content/hw1p2_student_data',
    'exp_load': 'layer_08_20220208-091953',
    'sample': 10000,
    'num_workers': 2,
    'model_path': MODEL_PATH,
    'file_out': 'submission_10.csv'
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Network08(args['context']).to(device)
pred_test(model, device, args)

Predict from model: layer_08_20220208-091953 (train loss=0.381520 | validation loss=0.391669 | validation accuracy=87.09% | epoch=99)


## Upload
Upload output file to Kaggle

In [ ]:
submission_file = args['file_out']
! kaggle competitions submit -c 11-785-s22-hw1p2 -f $submission_file -m "Submission 10"


100% 18.6M/18.6M [00:02<00:00, 6.57MB/s]
Successfully submitted to Frame-Level Speech Recognition